In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBModule, RGBDataModule

dm = RGBDataModule(batch_size=512, num_workers=20, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
model = RGBModule.load_from_checkpoint('checkpoints/baseline-val_loss=7.30568-epoch=7.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4bad48310>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1268, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff4bad48310>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1268, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdo

"backbone":         resnet18
"datamodule":       {'batch_size': 512, 'num_workers': 20, 'pin_memory': True}
"optimizer":        Adam
"optimizer_params": {'lr': 0.001}
"pretrained":       True
"trainer":          {'gpus': 1, 'max_epochs': 10, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x7ff47550c7f0>, 'enable_checkpointing': True, 'overfit_batches': 0, 'limit_train_batches': 100, 'callbacks': [<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff47550c820>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x7ff45027caf0>, <pytorch_lightning.callbacks.model_summary.ModelSummary object at 0x7ff45027cac0>, <pytorch_lightning.callbacks.gradient_accumulation_scheduler.GradientAccumulationScheduler object at 0x7ff45027cb20>]}

In [12]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda()
dl = dm.val_dataloader()
accs = []
for imgs, labels in tqdm(dl):
    preds = model.predict(imgs)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [00:21<00:00,  3.75it/s]


0.842739099859353

In [8]:
from tqdm import tqdm

model.cuda()
dl = dm.test_dataloader()
labels, observations = [], []
for imgs, observation_ids in tqdm(dl):
    preds = model.predict(imgs)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observations += observation_ids

100%|██████████| 72/72 [00:21<00:00,  3.36it/s]


In [6]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
29904,21369461,3072 3064 4999 2902 5025 5020 5247 5152 5074 5...
29890,21368630,1075 2071 2421 5366 1227 2268 5057 5045 2561 2...
10931,10757649,565 342 249 205 637 582 702 510 591 657 265 21...
28975,21268818,4940 5688 5 5580 5194 6171 5833 2723 5940 1900...
6685,10620751,397 692 49 471 1030 380 687 582 600 637 164 46...


In [7]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
25188,20864813,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18660,20148478,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
36359,22063536,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3949,10187584,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4003,10189550,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
9515,10453231,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
9746,10463974,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
16704,10796080,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
10265,10490711,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
32992,21698357,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
